# *MODELO FINAL*

## 1. Selección del Mejor Modelo
Tras evaluar distintos modelos (KNN, Árboles de Decisión, Regresión Logística, SVM, etc) se comprobó que el SVM con Kernel RBF y los hiperparámetros óptimos:
* C=10
* gamma = 'scale'
* kernel = 'rbf'

presenta el mejor desempeño en términos de Balanced Accuracy en el conjunto de validación interna. Esto lo convierte en nuestro candidato para el modelo final

## 2. Evaluación Outer y Estimación del Desempeño futuro
La estrategia de evaluación outer consiste en separar los datos en dos partes:

* Train (2/3): Para realizar el ajuste y selección de modelos
* Test (1/3): Para obtener una estimación real de rendimiento que tendría el modelo en una competición

Aunque durante la práctica se ha usado esta partición para evaluar el desempeño, en el paso final se reentrena al modelo usando todos los datos disponibles. 

## 4.Generación de Predicciones para la Competición
El siguiente paso es utilizar el modelo final para generar predicciones sobre el conjunto de datos de la competición. Para ello se debe aplicar el mismo preprocesado que a los datos de entrenamiento. El fichero resultante se guardará como predicciones.csv

In [12]:
import pandas as pd
import joblib
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# Importamos los datos de test y el modelo final
datos_test = pd.read_csv('./attrition_competition_10.csv.gz')
modelo_final = joblib.load('modelo_final.pkl')

print("Datos de test cargados y modelo final importado.")

# ---------------------------
# 4.1. Preprocesamiento Manual de los Datos de la Competición
# ---------------------------
# Para el conjunto de competición, eliminamos las mismas columnas que en el entrenamiento.
datos_test = datos_test.drop(columns=["EmployeeID", "EmployeeCount", "Over18", "StandardHours"])
datos_test = datos_test.drop(columns=["Attrition"], errors='ignore')

# Variables categóricas y ordinales (debes definirlas como en el entrenamiento)
categorical_features = ['Department', 'JobRole', 'EducationField']
ordinal_features = ['BusinessTravel', 'Gender', 'MaritalStatus']
valores_ordinales = [
    ['Non-Travel', 'Travel_Rarely', 'Travel_Frequently'],  # Para BusinessTravel
    ['Male', 'Female'],                                   # Para Gender
    ['Single', 'Married', 'Divorced']                     # Para MaritalStatus
]

# OneHotEncoding para variables categóricas
onehot_encoder = OneHotEncoder(drop='first', sparse_output=False)
categorical_encoded = onehot_encoder.fit_transform(datos_test[categorical_features])
categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=onehot_encoder.get_feature_names_out(categorical_features))

# OrdinalEncoding para variables ordinales
ordinal_encoder = OrdinalEncoder(categories=valores_ordinales)
ordinal_encoded = ordinal_encoder.fit_transform(datos_test[ordinal_features])
ordinal_encoded_df = pd.DataFrame(ordinal_encoded, columns=ordinal_features)

# Mantener el resto de las columnas (numéricas)
numeric_features = datos_test.drop(columns=categorical_features + ordinal_features).columns
numeric_df = datos_test[numeric_features].reset_index(drop=True)

# Concatenar todas las columnas preprocesadas
datos_test_preprocessed = pd.concat([categorical_encoded_df, ordinal_encoded_df, numeric_df], axis=1)

# Asegurarnos de que las columnas coincidan con las esperadas por el modelo
# Obtenemos las columnas esperadas directamente del modelo
expected_columns = modelo_final.best_estimator_.named_steps['knn'].feature_names_in_
datos_test_preprocessed = datos_test_preprocessed[expected_columns]

print("Completado - Preprocesamiento de Datos.")

# ---------------------------
# 4.2. Generar Predicciones
# ---------------------------
# Generamos predicciones con el modelo entrenado
y_pred_test = modelo_final.predict(datos_test_preprocessed)

# Creamos un DataFrame con las predicciones
df_predicciones = pd.DataFrame(y_pred_test, columns=["Predicción"])

print(df_predicciones["Predicción"].value_counts())  # Mostramos un mensaje por pantalla, predicciones realizadas
df_predicciones.to_csv("predicciones.csv", index=False)

print("Predicciones guardadas en 'predicciones.csv'")

Datos de test cargados y modelo final importado.


KeyError: 'knn'

# 5. Tarea Adicional


Para esta tarea adicional, hemos decicido utilizar la herramienta SHAP, para interpretar nuestro modelo final.

SHAP (SHapley Additive exPlanations) es una herramienta basada en la teoría de juegos que permite cuantificar la contribución de cada variable a la predicción de un modelo. Entre sus principales ventajas destacan:

* **Interpretabilidad Local y Global**: SHAP ofrece explicaciones tanto a nuvel individual (por cada predicción) como a nivel global (importancia de las variables a lo largo de todo el conjunto). Esto permite identificar cómo cada característica influye en cada decisión del modelo.

* **Consistencia y solidez**: Los valores de Shapley garantizan que la contribución de cada característica se mida de forma justa, lo que otorga una interpretación coherente y consistente.

* **Modelo-agnóstico**: A diferencia de otros métodos que se basan en la estructura interna del modelo (por ejemplo, la importancia basada en la impureza de los árboles en RandomForest), SHAP puede aplicarse a cualquier modelo.

* **Transparencia**: Integrar SHAP en el análisis del modelo permite detectar posibles sesgos o relaciones inesperadas entre variables, lo cual es fundamental para generar confianza en la toma de decisiones basadas en inteligencia artificial.

Realizar esta tarea extra con SHAP resulta especialmente interesante en comparación con construir otro modelo como un RandomForest o una red neuronal. Aunque estos modelos pueden ofrecer buenos resultados, la interpretación de sus resultados (por ejemplo, mediante la importancia de variables basada en el Gini o técnicas de regularización) suele ser menos precisa y profunda. En cambio, al interpretar el modelo SVM seleccionado con SHAP, se obtiene una explicación detallada y cuantitativa de la influencia de cada característica en la predicción, lo cual añade un valor extra al análisis, especialmente en contextos donde la explicabilidad es crucial.

## 5.1 Código para usar SHAP en nuestro modelo final

In [ ]:
import shap
import matplotlib.pyplot as plt
import pandas as pd

# Seleccionar una muestra representativa del conjunto preprocesado para el fondo (background)
# Reducir el tamaño del fondo para acelerar el cálculo
X_background = X_preprocessed.sample(20, random_state=42)

# Definir la función de predicción asegurándonos de que se pasan los nombres de las columnas
def model_predict(data):
    # Convertir a DataFrame si es necesario
    if not isinstance(data, pd.DataFrame):
        data = pd.DataFrame(data, columns=columnas_preprocesadas)
    return final_pipeline.predict_proba(data)[:, 1]

# Crear el explicador con KernelExplainer usando el fondo reducido
explainer = shap.KernelExplainer(model_predict, X_background)

# Seleccionar un subconjunto reducido de datos para explicar
X_explain = X_preprocessed.sample(10, random_state=42)

# Calcular los valores SHAP para el subconjunto seleccionado, limitando nsamples para acelerar el proceso
shap_values = explainer.shap_values(X_explain, nsamples=50)

# Visualizar el resumen de los valores SHAP para identificar la importancia de las variables
shap.summary_plot(shap_values, X_explain, feature_names=columnas_preprocesadas)
plt.show()

# Explicación local de una instancia en particular
i = 0  # índice de la instancia a explicar
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[i], X_explain.iloc[i], feature_names=columnas_preprocesadas)



ModuleNotFoundError: No module named 'shap'

La gráfica SHAP permite ver qué variables han influido más en las predicciones del modelo SVM sobre la variable Attrition. También muestra cómo valores altos o bajos de cada variable afectan individualmente a cada empleado.

En este caso, la variable más influyente es remainder__TotalWorkingYears, lo que indica que el número total de años trabajados es clave en la decisión. Empleados con más años suelen tener valores SHAP negativos, lo que reduce la probabilidad de abandono. También destacan variables como remainder__hrs, DistanceFromHome o TrainingTimesLastYear. Por ejemplo, empleados que viven lejos tienden a tener más probabilidad de dejar la empresa, mientras que recibir más formación puede asociarse a menor riesgo.

Las variables categóricas como el estado civil o el rol profesional (JobRole_Manager, MaritalStatus) también influyen, aunque en menor medida. Tener un cargo alto o estar casado parece relacionarse con mayor estabilidad.

En general, el gráfico muestra que la experiencia, la distancia al trabajo y otros factores numéricos son los que más afectan al resultado. Se observan patrones coherentes: valores altos en algunas variables empujan la predicción hacia Attrition = 1, mientras que en otras hacen lo contrario. Esto permite confiar más en el modelo y entender mejor sus decisiones.
